# Input Parameter Parsing

---

This tutorial covers various methods for parsing command-line arguments and handling user input in shell scripts, from basic parameter handling to advanced option parsing with validation.

Topics covered:
- Positional parameters
- getopts for option parsing
- Interactive input
- Input validation

## Basic Parameter Handling

### Positional Parameters

In [1]:
cat > /tmp/demo_params.sh << 'EOF'
#!/bin/bash
echo "Script name: $0"
echo "First parameter: $1"
echo "Second parameter: $2"
echo "Third parameter: $3"
echo "Number of parameters: $#"
echo "All parameters: $@"
echo "All parameters as single string: $*"
echo "Last parameter: ${!#}"
EOF

bash /tmp/demo_params.sh hello world 123

Script name: /tmp/demo_params.sh
First parameter: hello
Second parameter: world
Third parameter: 123
Number of parameters: 3
All parameters: hello world 123
All parameters as single string: hello world 123
Last parameter: 123
First parameter: hello
Second parameter: world
Third parameter: 123
Number of parameters: 3
All parameters: hello world 123
All parameters as single string: hello world 123
Last parameter: 123


### Parameter Validation

In [2]:
cat > /tmp/validate_params.sh << 'EOF'
#!/bin/bash
# Check if minimum number of parameters provided
if [ $# -lt 2 ]; then
    echo "Error: Insufficient parameters"
    echo "Usage: $0 <name> <age> [email]"
    exit 1
fi

name="$1"
age="$2"
email="${3:-not-provided}"

# Validate age is numeric
if ! [[ "$age" =~ ^[0-9]+$ ]]; then
    echo "Error: Age must be a number"
    exit 1
fi

# Validate age range
if [ "$age" -lt 0 ] || [ "$age" -gt 150 ]; then
    echo "Error: Age must be between 0 and 150"
    exit 1
fi

echo "Name: $name"
echo "Age: $age"
echo "Email: $email"
EOF

bash /tmp/validate_params.sh "John Doe" 30 john@example.com

Name: John Doe
Age: 30
Email: john@example.com
Age: 30
Email: john@example.com


### Shifting Parameters

In [3]:
cat > /tmp/shift_demo.sh << 'EOF'
#!/bin/bash
echo "Processing all parameters:"
while [ $# -gt 0 ]; do
    echo "Processing: $1"
    shift  # Move to next parameter
done
echo "All parameters processed"
EOF

bash /tmp/shift_demo.sh arg1 arg2 arg3

Processing all parameters:
Processing: arg1
Processing: arg2
Processing: arg3
All parameters processed
Processing: arg1
Processing: arg2
Processing: arg3
All parameters processed


## getopts - Standard Option Parsing

### Basic getopts Usage

In [4]:
cat > /tmp/getopts_basic.sh << 'EOF'
#!/bin/bash
# Initialize variables
verbose=false
output=""
help=false

# Parse options
while getopts "vo:h" opt; do
    case $opt in
        v)
            verbose=true
            ;;
        o)
            output="$OPTARG"
            ;;
        h)
            help=true
            ;;
        \?)
            echo "Invalid option: -$OPTARG" >&2
            exit 1
            ;;
        :)
            echo "Option -$OPTARG requires an argument" >&2
            exit 1
            ;;
    esac
done

# Shift processed options
shift $((OPTIND-1))

# Show help if requested
if [ "$help" = true ]; then
    echo "Usage: $0 [-v] [-o output_file] [-h] [files...]"
    echo "  -v: Verbose mode"
    echo "  -o: Output file"
    echo "  -h: Show this help"
    exit 0
fi

# Process remaining arguments
if [ "$verbose" = true ]; then
    echo "Verbose mode enabled"
    [ -n "$output" ] && echo "Output file: $output"
    echo "Remaining arguments: $@"
fi
EOF

echo "Example 1: Verbose mode"
bash /tmp/getopts_basic.sh -v -o result.txt file1.txt file2.txt

echo
echo "Example 2: Help"
bash /tmp/getopts_basic.sh -h

Example 1: Verbose mode
Verbose mode enabled
Output file: result.txt
Remaining arguments: file1.txt file2.txt
Verbose mode enabled
Output file: result.txt
Remaining arguments: file1.txt file2.txt


Example 2: Help
Example 2: Help
Usage: /tmp/getopts_basic.sh [-v] [-o output_file] [-h] [files...]
  -v: Verbose mode
  -o: Output file
  -h: Show this help
Usage: /tmp/getopts_basic.sh [-v] [-o output_file] [-h] [files...]
  -v: Verbose mode
  -o: Output file
  -h: Show this help


### Advanced getopts with Validation

In [5]:
cat > /tmp/file_processor.sh << 'EOF'
#!/bin/bash
# Default values
mode="copy"
destination=""
verbose=false

usage() {
    cat << USAGE
Usage: $0 [OPTIONS]

File processing utility

OPTIONS:
    -m MODE     Processing mode: copy, move, link (default: copy)
    -d DEST     Destination directory
    -v          Verbose output
    -h          Show this help
USAGE
}

# Parse options
while getopts "m:d:vh" opt; do
    case $opt in
        m)
            mode="$OPTARG"
            # Validate mode
            if [[ ! "$mode" =~ ^(copy|move|link)$ ]]; then
                echo "Error: Invalid mode '$mode'. Must be copy, move, or link." >&2
                exit 1
            fi
            ;;
        d)
            destination="$OPTARG"
            ;;
        v)
            verbose=true
            ;;
        h)
            usage
            exit 0
            ;;
    esac
done

shift $((OPTIND-1))

# Validation
if [ -z "$destination" ]; then
    echo "Error: Destination directory required (-d option)" >&2
    exit 1
fi

# Display configuration
echo "Mode: $mode"
echo "Destination: $destination"
echo "Verbose: $verbose"
EOF

bash /tmp/file_processor.sh -m copy -d /tmp/backup -v

Mode: copy
Destination: /tmp/backup
Verbose: true
Destination: /tmp/backup
Verbose: true


## Interactive Input

### Basic User Input

In [6]:
cat > /tmp/interactive_input.sh << 'EOF'
#!/bin/bash
# Simple input (simulated for notebook)
name="TestUser"
echo "Name (simulated): $name"

# Input with default value
country="USA"
echo "Country (default): $country"

# Demonstrate timeout concept
echo "Read with timeout would wait 5 seconds..."
echo "(Skipped in notebook environment)"
EOF

bash /tmp/interactive_input.sh

Name (simulated): TestUser
Country (default): USA
Read with timeout would wait 5 seconds...
(Skipped in notebook environment)
Country (default): USA
Read with timeout would wait 5 seconds...
(Skipped in notebook environment)


### Input Validation Functions

In [7]:
cat > /tmp/input_validation.sh << 'EOF'
#!/bin/bash

# Validate email address
validate_email() {
    local email="$1"
    local email_regex="^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
    
    if [[ "$email" =~ $email_regex ]]; then
        return 0
    else
        return 1
    fi
}

# Validate phone number
validate_phone() {
    local phone="$1"
    # Remove all non-digits
    local digits_only="${phone//[^0-9]/}"
    
    # Check if it has 10 digits
    if [ ${#digits_only} -eq 10 ]; then
        return 0
    else
        return 1
    fi
}

# Test email validation
emails=("user@example.com" "invalid.email" "test@domain.co.uk")
for email in "${emails[@]}"; do
    if validate_email "$email"; then
        echo "✓ Valid email: $email"
    else
        echo "✗ Invalid email: $email"
    fi
done

echo

# Test phone validation
phones=("555-123-4567" "12345" "(555) 123-4567")
for phone in "${phones[@]}"; do
    if validate_phone "$phone"; then
        echo "✓ Valid phone: $phone"
    else
        echo "✗ Invalid phone: $phone"
    fi
done
EOF

bash /tmp/input_validation.sh

✓ Valid email: user@example.com
✗ Invalid email: invalid.email
✓ Valid email: test@domain.co.uk

✓ Valid phone: 555-123-4567
✗ Invalid phone: 12345
✓ Valid phone: (555) 123-4567
✗ Invalid email: invalid.email
✓ Valid email: test@domain.co.uk

✓ Valid phone: 555-123-4567
✗ Invalid phone: 12345
✓ Valid phone: (555) 123-4567


### Menu Selection

In [8]:
cat > /tmp/menu_demo.sh << 'EOF'
#!/bin/bash

show_menu() {
    echo "Please select an option:"
    echo "1) Create new user"
    echo "2) Update existing user"
    echo "3) Delete user"
    echo "4) List all users"
    echo "5) Exit"
    echo
}

# Simulate menu (for notebook)
show_menu
choice=1
echo "Selected (simulated): $choice"

case $choice in
    1) echo "Action: Create new user" ;;
    2) echo "Action: Update existing user" ;;
    3) echo "Action: Delete user" ;;
    4) echo "Action: List all users" ;;
    5) echo "Action: Exit" ;;
    *) echo "Invalid choice" ;;
esac
EOF

bash /tmp/menu_demo.sh

Please select an option:
1) Create new user
2) Update existing user
3) Delete user
4) List all users
5) Exit

Selected (simulated): 1
Action: Create new user
1) Create new user
2) Update existing user
3) Delete user
4) List all users
5) Exit

Selected (simulated): 1
Action: Create new user


## Configuration File Parsing

In [9]:
# Create sample configuration file
cat << 'EOF' > /tmp/app.conf
# Application Configuration
database_host = localhost
database_port = 5432
database_name = myapp

# Logging
log_level = info
debug = false

# Connection settings
max_connections = 100
EOF

cat > /tmp/config_parser.sh << 'EOF'
#!/bin/bash

# Default configuration
declare -A config=(
    ["database_host"]="localhost"
    ["database_port"]="5432"
    ["log_level"]="info"
    ["debug"]="false"
    ["max_connections"]="100"
)

# Parse configuration file
parse_config() {
    local config_file="$1"
    
    while IFS= read -r line || [ -n "$line" ]; do
        # Skip empty lines and comments
        [[ "$line" =~ ^[[:space:]]*$ ]] && continue
        [[ "$line" =~ ^[[:space:]]*# ]] && continue
        
        # Parse key=value pairs
        if [[ "$line" =~ ^[[:space:]]*([a-zA-Z_][a-zA-Z0-9_]*)[[:space:]]*=[[:space:]]*(.*)[[:space:]]*$ ]]; then
            local key="${BASH_REMATCH[1]}"
            local value="${BASH_REMATCH[2]}"
            
            config["$key"]="$value"
            echo "Loaded: $key = $value"
        fi
    done < "$config_file"
}

echo "Parsing configuration..."
parse_config "/tmp/app.conf"

echo
echo "Current configuration:"
for key in "${!config[@]}"; do
    printf "  %-20s = %s\n" "$key" "${config[$key]}"
done
EOF

bash /tmp/config_parser.sh

Parsing configuration...
Loaded: database_host = localhost
Loaded: database_port = 5432
Loaded: database_name = myapp
Loaded: log_level = info
Loaded: debug = false
Loaded: max_connections = 100

Current configuration:
  max_connections      = 100
  debug                = false
  log_level            = info
  database_name        = myapp
  database_port        = 5432
  database_host        = localhost
Loaded: database_host = localhost
Loaded: database_port = 5432
Loaded: database_name = myapp
Loaded: log_level = info
Loaded: debug = false
Loaded: max_connections = 100

Current configuration:
  max_connections      = 100
  debug                = false
  log_level            = info
  database_name        = myapp
  database_port        = 5432
  database_host        = localhost


## Real-world Example: Backup Utility

In [10]:
cat > /tmp/backup_tool.sh << 'EOF'
#!/bin/bash

# Default configuration
source=""
destination=""
compression="gzip"
dry_run=false
verbose=false

usage() {
    cat << USAGE
BACKUP UTILITY v1.0

USAGE:
    $0 [OPTIONS] source destination

OPTIONS:
    -c TYPE     Compression: gzip, bzip2, xz, none (default: gzip)
    -n          Dry-run mode
    -v          Verbose output
    -h          Show this help

EXAMPLES:
    $0 -v /home/user /backup
    $0 -c bzip2 -n /var/www /backup
USAGE
}

# Parse options
while getopts "c:nvh" opt; do
    case $opt in
        c) compression="$OPTARG" ;;
        n) dry_run=true ;;
        v) verbose=true ;;
        h) usage; exit 0 ;;
        *) usage >&2; exit 1 ;;
    esac
done

shift $((OPTIND-1))

# Get positional arguments
source="$1"
destination="$2"

# Validation
if [ -z "$source" ] || [ -z "$destination" ]; then
    echo "Error: Source and destination required" >&2
    usage >&2
    exit 1
fi

# Display configuration
echo "Backup Configuration:"
echo "  Source: $source"
echo "  Destination: $destination"
echo "  Compression: $compression"
echo "  Dry-run: $dry_run"
echo "  Verbose: $verbose"

if [ "$dry_run" = true ]; then
    echo "DRY RUN MODE - No actual backup performed"
fi
EOF

echo "Example 1:"
bash /tmp/backup_tool.sh -v -c bzip2 /home/user /backup

echo
echo "Example 2:"
bash /tmp/backup_tool.sh -n /var/www /backup

Example 1:
Backup Configuration:
  Source: /home/user
  Destination: /backup
  Compression: bzip2
  Dry-run: false
  Verbose: true
Backup Configuration:
  Source: /home/user
  Destination: /backup
  Compression: bzip2
  Dry-run: false
  Verbose: true


Example 2:
Example 2:
Backup Configuration:
  Source: /var/www
  Destination: /backup
  Compression: gzip
  Dry-run: true
  Verbose: false
DRY RUN MODE - No actual backup performed
Backup Configuration:
  Source: /var/www
  Destination: /backup
  Compression: gzip
  Dry-run: true
  Verbose: false
DRY RUN MODE - No actual backup performed


## Exercise

Create a script called `system_monitor.sh` that accepts various input parameters:

**Command-line options:**
- `-i SECONDS`: Monitoring interval (default: 5)
- `-d MINUTES`: Duration (default: 60)
- `-o FILE`: Output file
- `-v`: Verbose mode
- `-h`: Help

The script should:
1. Parse command-line options
2. Validate numeric inputs
3. Support a configuration file (optional)
4. Provide helpful error messages

## Cleanup

In [11]:
# Clean up temporary files
rm -f /tmp/*.sh /tmp/app.conf
echo "Temporary files cleaned up"

Temporary files cleaned up
